In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
import time
import chess
import matplotlib.pyplot as plt 
import re
from itertools import combinations 
import operator
import statistics as stats
import math
import chess.engine
import torch.optim as optim
from stockfish import Stockfish

In [2]:
engine = chess.engine.SimpleEngine.popen_uci('C:/Users/Justin/Desktop/currentworks/miningproject/stockfish-11-win/Windows/stockfish_20011801_x64')
stockfish = Stockfish('C:/Users/Justin/Desktop/currentworks/miningproject/stockfish-11-win/Windows/stockfish_20011801_x64')

In [3]:
lichess_games = pd.read_csv("E:/Users/Justin/Desktop/Course Folders 2020/DSC440Mining/games.csv")

In [4]:
lichess = lichess_games.drop(columns = ['id', 'rated', 'created_at', 'last_move_at', 'increment_code', 'white_id', 'black_id', 'opening_eco'])

In [5]:
highgames = lichess[lichess['white_rating'] > 2000]
highgames = highgames[highgames['black_rating'] > 2000]

In [6]:
highgames_reduced = highgames[highgames['turns'] > 20]

In [7]:
def get_training_list():
    fen_list = []
    next_move = []
    index_list = []
    
    for i in range(3):
        x = highgames_reduced.sample(n=1)
        random_range = random.choice(range(10, int(x['turns']) - 5, 2))
        board = chess.Board()
        for j in list(x['moves'])[0].split(' ')[:random_range]:
            #print(j)
            board.push_san(j)
        #index_list.append()
        fen_list.append(board.fen())
        next_move.append(list(x['moves'])[0].split(' ')[random_range])
        
    for i in range(3):
        x = highgames_reduced.sample(n=1)
        random_range = random.choice(range(11, int(x['turns']) - 6, 2))
        board = chess.Board()
        for j in list(x['moves'])[0].split(' ')[:random_range]:
            #print(j)
            board.push_san(j)
        #index_list.append()
        fen_list.append(board.fen())
        next_move.append(list(x['moves'])[0].split(' ')[random_range])
    
    return fen_list, next_move

In [25]:
fen_list, next_moves = get_training_list()

In [9]:
pawntable = [ 0,  0,  0,  0,  0,  0,  0,  0,
             50, 50, 50, 50, 50, 50, 50, 50,
             10, 10, 20, 30, 30, 20, 10, 10,
             5,  5, 10, 25, 25, 10,  5,  5,
             0,  0,  0, 20, 20,  0,  0,  0,
             5, -5,-10,  0,  0,-10, -5,  5,
             5, 10, 10,-20,-20, 10, 10,  5,
             0,  0,  0,  0,  0,  0,  0,  0]

knighttable = [-50,-40,-30,-30,-30,-30,-40,-50,
               -40,-20,  0,  0,  0,  0,-20,-40,
               -30,  0, 10, 15, 15, 10,  0,-30,
               -30,  5, 15, 20, 20, 15,  5,-30,
               -30,  0, 15, 20, 20, 15,  0,-30,
               -30,  5, 10, 15, 15, 10,  5,-30,
               -40,-20,  0,  5,  5,  0,-20,-40,
               -50,-40,-30,-30,-30,-30,-40,-50]

bishoptable = [-20,-10,-10,-10,-10,-10,-10,-20,
               -10,  0,  0,  0,  0,  0,  0,-10,
               -10,  0,  5, 10, 10,  5,  0,-10,
               -10,  5,  5, 10, 10,  5,  5,-10,
               -10,  0, 10, 10, 10, 10,  0,-10,
               -10, 10, 10, 10, 10, 10, 10,-10,
               -10,  5,  0,  0,  0,  0,  5,-10,
               -20,-10,-10,-10,-10,-10,-10,-20]

rooktable = [0,  0,  0,  0,  0,  0,  0,  0,
             5, 10, 10, 10, 10, 10, 10,  5,
             -5,  0,  0,  0,  0,  0,  0, -5,
             -5,  0,  0,  0,  0,  0,  0, -5,
             -5,  0,  0,  0,  0,  0,  0, -5,
             -5,  0,  0,  0,  0,  0,  0, -5,
             -5,  0,  0,  0,  0,  0,  0, -5,
             0,  0,  0,  5,  5,  0,  0,  0]

queentable = [-20,-10,-10, -5, -5,-10,-10,-20,
              -10,  0,  0,  0,  0,  0,  0,-10,
              -10,  0,  5,  5,  5,  5,  0,-10,
              -5,  0,  5,  5,  5,  5,  0, -5,
              0,  0,  5,  5,  5,  5,  0, -5,
              -10,  5,  5,  5,  5,  5,  0,-10,
              -10,  0,  5,  0,  0,  0,  0,-10,
              -20,-10,-10, -5, -5,-10,-10,-20]

kingtable = [-30,-40,-40,-50,-50,-40,-40,-30,
             -30,-40,-40,-50,-50,-40,-40,-30,
             -30,-40,-40,-50,-50,-40,-40,-30,
             -30,-40,-40,-50,-50,-40,-40,-30,
             -20,-30,-30,-40,-40,-30,-30,-20,
             -10,-20,-20,-20,-20,-20,-20,-10,
             20, 20,  0,  0,  0,  0, 20, 20,
             20, 30, 10,  0,  0, 10, 30, 20]

In [23]:
def minimax_move(board, depth = 0, move_weights = None, count = 0):
    
    if move_weights == None:
        move_weights = {i : 0 for i in board.legal_moves}
    for i in board.legal_moves:
        count += 1
        #if depth == 0:
           #print(i)
        
        board_copy = board.copy()

        board_copy.push(i)

        move_weights[i] += evaluation1(board_copy)
        #move_weights[i] += evaluation2(board_copy)

        if board.is_checkmate() and depth%2 == 0:
            move_weights[i] += 1_000_000
        elif board.is_checkmate() and depth%2 == 1:
            move_weights[i] -= 1_000_000
        
        if board.is_stalemate() or board.is_insufficient_material():
            move_weights[i] = 0
            
        current_move_weights = {i : 0 for i in board_copy.legal_moves}
        if depth != 3 and not board.is_game_over() and current_move_weights != {}:
            
            _, val = minimax_move(board_copy, depth + 1, current_move_weights, count)
            move_weights[i] += val
            
    if depth%2 == 0:
        if depth == 0:
            print('moves evaluated: ', count)
            print(move_weights)
        return max(move_weights, key=move_weights.get), max(move_weights.values())
    else:
        return min(move_weights, key=move_weights.get), min(move_weights.values())
    

In [11]:
def evaluation1(board):
    
    piece_weights = {'P': 1, 'N': 3, 'B': 4, 'R': 5, 'Q': 9, 'K': 50}
    
    total_evaluation = 0
    
    ### pieces
    pawns_difference = piece_weights['P'] * (len(board.pieces(chess.PAWN, chess.WHITE)) - len(board.pieces(chess.PAWN, chess.BLACK)))
    knight_difference = piece_weights['N'] * (len(board.pieces(chess.KNIGHT, chess.WHITE)) - len(board.pieces(chess.KNIGHT, chess.BLACK)))
    bishop_difference = piece_weights['B'] * (len(board.pieces(chess.BISHOP, chess.WHITE)) - len(board.pieces(chess.BISHOP, chess.BLACK)))
    rook_difference = piece_weights['R'] * (len(board.pieces(chess.ROOK, chess.WHITE)) - len(board.pieces(chess.ROOK, chess.BLACK)))
    queen_difference = piece_weights['Q'] * (len(board.pieces(chess.QUEEN, chess.WHITE)) - len(board.pieces(chess.QUEEN, chess.BLACK)))

    piece_evaluation = pawns_difference + knight_difference + bishop_difference + rook_difference + queen_difference
    
    pawns = 0
    for i in list(board.pieces(chess.PAWN, chess.WHITE)):   
        pawns += pawntable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.PAWN, chess.BLACK)):
        pawns -= pawntable[i]
    
    knights = 0
    for i in list(board.pieces(chess.KNIGHT, chess.WHITE)):
        knights += knighttable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.KNIGHT, chess.BLACK)):
        knights -= knighttable[i]
    
    bishops = 0
    for i in list(board.pieces(chess.BISHOP, chess.WHITE)):
        bishops += bishoptable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.BISHOP, chess.BLACK)):
        bishops -= bishoptable[i]
    
    rooks = 0
    for i in list(board.pieces(chess.ROOK, chess.WHITE)):
        rooks += rooktable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.ROOK, chess.BLACK)):
        rooks -= rooktable[i]
    
    queen = 0
    for i in list(board.pieces(chess.QUEEN, chess.WHITE)):
        queen += queentable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.QUEEN, chess.BLACK)):
        queen -= queentable[i]
        
    king = 0
    for i in list(board.pieces(chess.KING, chess.WHITE)):
        king += kingtable[chess.square_mirror(i)]
    for i in list(board.pieces(chess.KING, chess.BLACK)):
        king -= kingtable[i]
        
    board_evaluation = pawns + knights + bishops + rooks + queen + king
    
    total_evaluation = board_evaluation + piece_evaluation
    
    return total_evaluation

In [12]:
whitequeentable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whiteqtable.ckpt')
whitebishoptable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whitebtable.ckpt')
whiterooktable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whitertable.ckpt')
whiteknighttable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whitentable.ckpt')
whitekingtable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whitektable.ckpt')
whitepawntable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/whiteptable.ckpt')

In [13]:
blackqueentable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackqtable.ckpt')
blackbishoptable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackbtable.ckpt')
blackrooktable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackrtable.ckpt')
blackknighttable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackntable.ckpt')
blackkingtable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackktable.ckpt')
blackpawntable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/blackptable.ckpt')

In [14]:
whitequeenertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhiteq.ckpt')
whitebishopertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhiteb.ckpt')
whiterookertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhiter.ckpt')
whiteknightertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhiten.ckpt')
whitekingertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhitek.ckpt')
whitepawnertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorswhitep.ckpt')

In [15]:
blackqueenertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackq.ckpt')
blackbishopertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackb.ckpt')
blackrookertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackr.ckpt')
blackknightertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackn.ckpt')
blackkingertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackk.ckpt')
blackpawnertable = torch.load('C:/Users/Justin/Desktop/currentworks/miningproject/saving/errorsblackp.ckpt')

In [16]:
whitequeentable2 = whitequeentable * whitequeenertable /torch.max(whitequeenertable)
whitebishoptable2 = whitebishoptable * whitebishopertable /torch.max(whitebishopertable)
whiterooktable2 = whiterooktable * whiterookertable /torch.max(whiterookertable)
whiteknighttable2 = whiteknighttable * whiteknightertable /torch.max(whiteknightertable)
whitekingtable2 = whitekingtable * whitekingertable 
whitepawntable2 = whitepawntable * whitepawntable/10

In [17]:
blackqueentable2 = blackqueentable * blackqueenertable / torch.max(blackqueenertable)
blackbishoptable2 = blackbishoptable * blackbishopertable / torch.max(blackbishopertable)
blackrooktable2 = blackrooktable * blackrookertable /torch.max(blackrookertable)
blackknighttable2 = blackknighttable * blackknightertable /torch.max(blackknightertable)
blackkingtable2 = blackkingtable * blackkingertable
blackpawntable2 = blackpawntable * blackpawntable/10

In [18]:
def evaluation2(board):
    
    piece_weights = {'P': 1, 'N': 3, 'B': 4, 'R': 5, 'Q': 9, 'K': 50}
    
    total_evaluation = 0
    
    ### pieces
    pawns_difference = piece_weights['P'] * (len(board.pieces(chess.PAWN, chess.WHITE)) - len(board.pieces(chess.PAWN, chess.BLACK)))
    knight_difference = piece_weights['N'] * (len(board.pieces(chess.KNIGHT, chess.WHITE)) - len(board.pieces(chess.KNIGHT, chess.BLACK)))
    bishop_difference = piece_weights['B'] * (len(board.pieces(chess.BISHOP, chess.WHITE)) - len(board.pieces(chess.BISHOP, chess.BLACK)))
    rook_difference = piece_weights['R'] * (len(board.pieces(chess.ROOK, chess.WHITE)) - len(board.pieces(chess.ROOK, chess.BLACK)))
    queen_difference = piece_weights['Q'] * (len(board.pieces(chess.QUEEN, chess.WHITE)) - len(board.pieces(chess.QUEEN, chess.BLACK)))

    piece_evaluation = pawns_difference + knight_difference + bishop_difference + rook_difference + queen_difference
    
    pawns = 0
    for i in list(board.pieces(chess.PAWN, chess.WHITE)):   
        pawns += whitepawntable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.PAWN, chess.BLACK)):
        pawns -= blackpawntable2[i]
    
    knights = 0
    for i in list(board.pieces(chess.KNIGHT, chess.WHITE)):
        knights += whiteknighttable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.KNIGHT, chess.BLACK)):
        knights -= blackknighttable2[i]
    
    bishops = 0
    for i in list(board.pieces(chess.BISHOP, chess.WHITE)):
        bishops += whitebishoptable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.BISHOP, chess.BLACK)):
        bishops -= blackbishoptable2[i]
    
    rooks = 0
    for i in list(board.pieces(chess.ROOK, chess.WHITE)):
        rooks += whiterooktable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.ROOK, chess.BLACK)):
        rooks -= blackrooktable2[i]
    
    queen = 0
    for i in list(board.pieces(chess.QUEEN, chess.WHITE)):
        queen += whitequeentable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.QUEEN, chess.BLACK)):
        queen -= blackqueentable2[i]
        
    king = 0
    for i in list(board.pieces(chess.KING, chess.WHITE)):
        king += whitekingtable2[chess.square_mirror(i)]
    for i in list(board.pieces(chess.KING, chess.BLACK)):
        king -= blackkingtable2[i]
        
    board_evaluation = pawns + knights + bishops + rooks + queen + king
    
    total_evaluation = board_evaluation + piece_evaluation
    
    return total_evaluation

In [19]:
'''
board = chess.Board("1k1r4/pp1b1R2/3q2pp/4p3/2B5/4Q3/PPP2B2/2K5 b - - 0 1")
start = time.time()
print(minimax_move(board))
end = time.time()
print(end - start, 'seconds')
'''

'\nboard = chess.Board("1k1r4/pp1b1R2/3q2pp/4p3/2B5/4Q3/PPP2B2/2K5 b - - 0 1")\nstart = time.time()\nprint(minimax_move(board))\nend = time.time()\nprint(end - start, \'seconds\')\n'

In [24]:
times = []
for i in fen_list:
    board = chess.Board(i)
    start = time.time()
    move, val = minimax_move(board)
    print(move, val)
    info = engine.analyse(board, chess.engine.Limit(time=.1), root_moves = [move])   
    stockfish.set_fen_position(board.fen())
    print(stockfish.get_best_move())
    print(info['score'])
    end = time.time()
    times.append(end - start)
    print(end - start, 'seconds')
    print('\n')


moves evaluated:  43
{Move.from_uci('h3e6'): -162, Move.from_uci('h3f5'): -156, Move.from_uci('h3g4'): -179, Move.from_uci('h3g2'): -181, Move.from_uci('e3a7'): -250, Move.from_uci('e3h6'): -188, Move.from_uci('e3b6'): -239, Move.from_uci('e3g5'): -214, Move.from_uci('e3c5'): -219, Move.from_uci('e3f4'): -219, Move.from_uci('e3e4'): -243, Move.from_uci('e3d4'): -219, Move.from_uci('e3g3'): -197, Move.from_uci('e3f3'): -219, Move.from_uci('e3d3'): -192, Move.from_uci('e3c3'): -204, Move.from_uci('e3b3'): -219, Move.from_uci('e3f2'): -212, Move.from_uci('e3d2'): -212, Move.from_uci('e3c1'): -256, Move.from_uci('h2g3'): -337, Move.from_uci('h2g2'): -219, Move.from_uci('h2h1'): -219, Move.from_uci('g1g7'): -150, Move.from_uci('g1g6'): -219, Move.from_uci('g1g5'): -219, Move.from_uci('g1g4'): -219, Move.from_uci('g1g3'): -219, Move.from_uci('g1g2'): -219, Move.from_uci('g1h1'): -219, Move.from_uci('f1f7'): -150, Move.from_uci('f1f6'): -219, Move.from_uci('f1f5'): -219, Move.from_uci('f1f4')

In [21]:
times2 = []
for i in fen_list:
    board = chess.Board(i)
    start = time.time()
    move, val = minimax_move(board)
    print(move, val)
    info = engine.analyse(board, chess.engine.Limit(time=.1), root_moves = [move])   
    stockfish.set_fen_position(board.fen())
    print(stockfish.get_best_move())
    print(info['score'])
    end = time.time()
    times2.append(end - start)
    print(end - start, 'seconds')
    print('\n')

moves evaluated:  43
{Move.from_uci('h3e6'): tensor(-2260.3970), Move.from_uci('h3f5'): tensor(-2324.4138), Move.from_uci('h3g4'): tensor(-3512.1133), Move.from_uci('h3g2'): tensor(-4082.9431), Move.from_uci('e3a7'): tensor(-2314.4272), Move.from_uci('e3h6'): tensor(-6733.8784), Move.from_uci('e3b6'): tensor(-2316.9524), Move.from_uci('e3g5'): tensor(-2330.1172), Move.from_uci('e3c5'): tensor(-2312.3921), Move.from_uci('e3f4'): tensor(-2266.4915), Move.from_uci('e3e4'): tensor(-2262.5598), Move.from_uci('e3d4'): tensor(-2318.7681), Move.from_uci('e3g3'): tensor(-2314.8330), Move.from_uci('e3f3'): tensor(-2265.4653), Move.from_uci('e3d3'): tensor(-2266.5591), Move.from_uci('e3c3'): tensor(-2316.7659), Move.from_uci('e3b3'): tensor(-2313.2700), Move.from_uci('e3f2'): tensor(-2267.9380), Move.from_uci('e3d2'): tensor(-2313.6753), Move.from_uci('e3c1'): tensor(-2317.3164), Move.from_uci('h2g3'): tensor(-3061.1731), Move.from_uci('h2g2'): tensor(-1170.1731), Move.from_uci('h2h1'): tensor(-2

In [22]:
##original

##1
##calculated c3d4 with 119
##correct move is e4d4
##score of calculated move is -1130
##calculated of correct move 10

##2
##calculated d3c2 with -2
##correct move is h4e4
##score of calculated move is -727
##calculated of correct move -72

##3
##calculated c3d4 with 131
##correct move is b2a4
##score of calculated move is -1261
##calculated of correct move -41

##4
##calculated d7b8 with 90
##correct move is d5h5
##score of calculated move is -810
##calculated of correct move -4

##5
##calculated c6b8 with 114
##correct move is h5f3
##score of calculated move is -416
##calculated of correct move -102

##6
##calculated c5a6 with 181
##correct move is d6a5
##score of calculated move is -6
##calculated of correct move -138

In [ ]:
##data influence
##1
##calculated h2g2 with -1170.1731
##correct move is e3g3
##score of calculated move is 151

##2
##calculated g4f3 with -2100.3130
##correct move is g4f3
##score of calculated move is 1335

##3
##calculated e2h5 with -1950.0792
##correct move is c3d5
##score of calculated move is -1375

##4
##calculated e8e7 with 890.2587
##correct move is b8c6
##score of calculated move is -387

##5
##calculated e8g8 with 2453.1362
##correct move is d7d5
##score of calculated move is -648

##6
##calculated g6g2 with 6165.5762
##correct move is e6d5
##score of calculated move is -1914

In [ ]:
##original
##1
##calculated g1g7 with -150
##correct move is e3g3
##score of calculated move is 0

##2
##calculated g4h3 with 118
##correct move is g4f3
##score of calculated move is 1370

##3
##calculated c1f4 with 83
##correct move is c3d5
##score of calculated move is -471

##4
##calculated b7b5 with 260
##correct move is b8c6
##score of calculated move is -429

##5
##calculated f6g8 with 336
##correct move is d7d5
##score of calculated move is -789

##6
##calculated g8f7 with 113
##correct move is e6d5
##score of calculated move is -895